Dependencies | Version
--- | ---
SatPy | 0.25.1
PyResample | 1.17.0
PySpectral | 0.10.4
Trollimage | 1.14.0
PyKdtree | 1.3.4
XArray | 0.16.1
Dask | 2.30.0
ImageIO | 2.9.0
ffmpeg | 4.3.1


# GOES-16 ABI - True Color Animation - Hurricane Florence



This example shows how to take multiple time steps of ABI data and combine them in to a single mp4 movie file as an animation. The example uses 100 frames of mesoscale images from the GOES-16 ABI instrument on September 11th, 2018 from 1300Z to 1440Z with one image every minute. The "%%time" magic command is used in jupyter notebook to time the entire cell's execution. This example took about 8 minutes and 30 seconds on a 2015 macbook using 8 dask threaded workers.

The movie produced has been hosted on twitter: https://twitter.com/PyTrollOrg/status/1039555399433834497

Full resolution video: https://www.ssec.wisc.edu/~davidh/satpy/true_color_20180911_130021.mp4

The frames in the movie are 500m resolution true color images with solar zenith and rayleigh scattering corrections applied. A ratio sharpening technique was also used to bring out more detail from the lower resolution input bands. All of this is pre-configured for the ABI 'true_color' composite in satpy and is done from python.

In [ ]:
%%time
import os
import warnings
import logging
from satpy import Scene
from glob import glob
from satpy import MultiScene
logging.basicConfig(level=logging.ERROR)  # don't show warnings
warnings.simplefilter('ignore')  # don't show warnings

BASE_DIR = '/data/satellite/abi/20180911_florence_100'

mscn = MultiScene.from_files(glob(os.path.join(BASE_DIR, 'OR*-RadM1*.nc')), reader='abi_l1b')
mscn.load(['true_color'])
new_mscn = mscn.resample(resampler='native')
new_mscn.save_animation('{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=5)

In [ ]:
from IPython.display import Video
Video('true_color_20180911_130021.mp4', width=480, height=480)

# Advanced Usage

It is also possible to use dask in a multiprocess environment to compute multiple animation frames at once. We do this by first creating a dask `Client` object which will create external dask workers for us. We then pass this client object to `save_animation` along with a `batch_size`. This tells Satpy to compute `16` frames at a time by using the parallel multiprocess workers created by dask. Without this, Satpy will default to computing one frame at a time. Depending on your system, this can cut off minutes of processing time.

Note: The creation of the `Client` below includes making a dashboard available for viewing tasks as they are worked on by dask.

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
%%time
import os
import warnings
import logging
from satpy import Scene
from glob import glob
from satpy import MultiScene
logging.basicConfig(level=logging.ERROR)  # don't show warnings
warnings.simplefilter('ignore')  # don't show warnings

BASE_DIR = '/data/satellite/abi/20180911_florence_100'

mscn = MultiScene.from_files(glob(os.path.join(BASE_DIR, 'OR*-RadM1*.nc')), reader='abi_l1b')
mscn.load(['true_color'])
new_mscn = mscn.resample(resampler='native')
new_mscn.save_animation('{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=5, batch_size=16, client=client)